In [2]:
import pandas as pd
import numpy as np
import keras 
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1_l2
import os

In [3]:
dev = pd.read_csv("D:/Course content/Minor Project/TDT_Data/Validation.csv")
dev.rename( columns={'Unnamed: 0':'0'}, inplace=True )
dev.drop('0', axis = 1)

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
0,0.409480,0.330053,0.588166,-0.058139,-0.359565,0.061654,0.783157,0.434004,0.049200,0.890888,...,0.892950,-0.767745,0.439113,0.365136,0.573477,0.695570,-0.724616,0.458630,0.611607,0.826896
1,0.533911,0.643805,0.901475,0.143997,0.013035,-0.379633,0.730578,0.515096,0.347368,0.935897,...,0.787014,-0.724356,0.284720,0.818754,0.730015,0.524863,-0.262078,0.392839,0.289762,0.825478
2,0.509951,0.666751,0.907186,0.363035,0.070572,-0.687706,0.727476,0.427720,0.543863,0.912617,...,0.538350,0.519184,0.546229,0.941528,0.787032,-0.364118,0.216141,0.176713,0.060211,0.789942
3,0.240762,0.831359,0.661822,0.248816,0.096252,-0.365287,0.473186,0.073838,0.828981,0.782181,...,-0.526119,0.647022,0.264069,0.884479,0.790552,-0.740754,0.134361,0.530370,0.342484,0.852868
4,0.187797,0.760324,0.483309,0.194137,0.020227,0.496607,0.460802,0.076955,0.879658,0.543056,...,-0.733004,0.453606,0.239479,0.868984,0.641466,-0.804020,0.340424,0.616168,0.640221,0.776289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55870,0.252430,-0.371339,0.337835,-0.069046,-0.220030,0.174933,0.475429,0.192603,0.816235,-0.536560,...,0.649733,0.813218,-0.451405,0.396054,0.541864,0.693254,-0.839364,0.218996,-0.412561,0.511329
55871,0.651506,-0.681530,0.853833,0.002269,-0.028894,0.440940,0.597065,-0.161940,0.099349,0.674637,...,0.248615,0.859941,-0.517496,0.586106,-0.085536,0.527270,-0.477805,0.318596,0.290935,0.737237
55872,0.357015,-0.478252,0.884538,0.095827,-0.070810,0.753470,0.732722,0.139935,-0.613398,0.642679,...,-0.005278,-0.265326,0.337169,-0.170902,-0.483197,-0.478460,0.013249,-0.413465,0.340715,0.650361
55873,0.323493,-0.257720,0.808529,-0.053447,0.058645,0.378627,0.448387,0.528075,-0.796151,0.662209,...,0.290560,-0.432021,0.435367,0.365950,0.096495,-0.820756,0.009772,-0.566893,0.428987,0.718649


In [4]:
dev_X = dev.drop(['0','21'], axis = 1)
dev_y = dev['21'].copy()
dev_X.to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Validation_X.csv')
dev_y.to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Validation_y.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [5]:
train = pd.read_csv("D:/Course content/Minor Project/TDT_Data/Train.csv")
train.rename( columns={'Unnamed: 0':'d'}, inplace=True )
train_X = train.drop(['d','20'], axis = 1)
train_X.to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Train_X.csv')

In [6]:
train_y = train['20'].copy()
train_y.to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Train_Y.csv')
pd.DataFrame(train_y).to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Train_Y.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [7]:
dev_X = dev_X.loc[:, ~dev_X.columns.str.contains('^Unnamed')]
train_X = train_X.loc[:, ~dev_X.columns.str.contains('^Unnamed')]

In [8]:
STEP = 20
_dev_X = np.asarray(dev_X).reshape((int(1117500/STEP), 20 , 1))
_dev_Y = np.asarray(dev_y).reshape(int(1117500/STEP), 1)
_train_X = np.asarray(train_X).reshape((int(1117500/STEP), 20, 1))
_train_Y = np.asarray(train_y).reshape(int(1117500/STEP), 1)

In [9]:
test = pd.read_csv("D:/Course content/Minor Project/TDT_Data/Test1.csv")
test.rename( columns={'Unnamed: 0':'d'}, inplace=True )
test_X = test.drop(['d','22'], axis = 1)

In [10]:
test_y = test['22'].copy()
test_X.to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Test_X.csv')
test_y.to_csv('D:/Course content/Minor Project/TDT_Data/X_Y_Split/Test_Y.csv')
_test_X = np.asarray(test_X).reshape((int(1117500/STEP), 20, 1))
_test_Y = np.asarray(test_y).reshape(int(1117500/STEP), 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
from keras.layers import Flatten
#define custom activation
class Double_Tanh(Activation):
    def __init__(self, activation, **kwargs):
        super(Double_Tanh, self).__init__(activation, **kwargs)
        self.__name__ = 'double_tanh'

def double_tanh(x):
    return (K.tanh(x))

get_custom_objects().update({'double_tanh':Double_Tanh(double_tanh)})

# Model Generation
model = Sequential()
model.add(Dense(55875, input_shape=(20, 1), activation='double_tanh'))
model.add(Flatten())
model.add(Dense(1))
# model.add(Activation(double_tanh))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

print(model.metrics_names)
# Fitting the Model
model_scores = {}
score_list =[]
Reg = False
d = 'MLP'

if Reg :
    d += '_with_reg'

epoch_num=1
for _ in range(5):

    # train the model
    m = model.fit(_train_X, _train_Y, epochs=1, batch_size=500, shuffle=True)
    print('history :', m.history)
    score_list.append(m.history)

    # test the model
    score_train = model.evaluate(_train_X, _train_Y)
    score_dev = model.evaluate(_dev_X, _dev_Y)
    score_test1 = model.evaluate(_test_X, _test_Y)

    print('train set score : mse - ' + str(score_train[1]) +' / mae - ' + str(score_train[2]))
    print('dev set score : mse - ' + str(score_dev[1]) +' / mae - ' + str(score_dev[2]))
    print('test set score : mse - ' + str(score_test1[1]) +' / mae - ' + str(score_test1[2]))
    df = pd.read_csv("D:/Course content/Minor Project/models/"+d+".csv")
    train_mse = list(df['TRAIN_MSE'])
    dev_mse = list(df['DEV_MSE'])
    test1_mse = list(df['TEST1_MSE'])


    train_mae = list(df['TRAIN_MAE'])
    dev_mae = list(df['DEV_MAE'])
    test1_mae = list(df['TEST1_MAE'])

    # append new data
    train_mse.append(score_train[1])
    dev_mse.append(score_dev[1])
    test1_mse.append(score_test1[1])

    train_mae.append(score_train[2])
    dev_mae.append(score_dev[2])
    test1_mae.append(score_test1[2])

    # organize newly created score dataset
    model_scores['TRAIN_MSE'] = train_mse
    model_scores['DEV_MSE'] = dev_mse
    model_scores['TEST1_MSE'] = test1_mse

    model_scores['TRAIN_MAE'] = train_mae
    model_scores['DEV_MAE'] = dev_mae
    model_scores['TEST1_MAE'] = test1_mae

    # save newly created score dataset
    model_scores_df = pd.DataFrame(model_scores)
    model_scores_df.to_csv("D:/Course content/Minor Project/models/"+d+".csv")

[]
112/112 [==============================] - 311s 3s/step - loss: 0.3364 - mse: 0.3364 - mae: 0.4571
history : {'loss': [0.3364291489124298], 'mse': [0.3364291489124298], 'mae': [0.45714402198791504]}
1747/1747 [==============================] - 133s 76ms/step - loss: 0.2565 - mse: 0.2565 - mae: 0.4462
train set score : mse - 0.22165408730506897 / mae - 0.40185749530792236
dev set score : mse - 0.2528919279575348 / mae - 0.43856319785118103
test set score : mse - 0.25647401809692383 / mae - 0.44623422622680664
112/112 [==============================] - 230s 2s/step - loss: 0.2215 - mse: 0.2215 - mae: 0.4006
history : {'loss': [0.22149614989757538], 'mse': [0.22149614989757538], 'mae': [0.40055176615715027]}
1747/1747 [==============================] - 139s 80ms/step - loss: 0.2564 - mse: 0.2564 - mae: 0.4469
train set score : mse - 0.22105346620082855 / mae - 0.4012467563152313
dev set score : mse - 0.25393417477607727 / mae - 0.43966421484947205
test set score : mse - 0.2564434409141

In [12]:
import csv
with open('D:/Course content/Minor Project/models/MLP.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["TRAIN_MSE", "DEV_MSE", "TEST1_MSE", "TRAIN_MAE","DEV_MAE", "TEST1_MAE"])